<a href="https://colab.research.google.com/github/LashawnFofung/RAG-Pipelines/blob/main/Gradio/Task_Gradio_Chatbot_without_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gradio Chatbot**

**GOAL:** A document Q&A interface with a sleek monochrome theme.

<br>

**PDF Research Buddy: Project Submission**

This notebook contains a Gradio-based chatbot designed to analyze PDF documents using a monochrome theme and a custom "Analysis Mode" feature.

<br>

**FEATURES:**
1. Mode Selection (Dropdown) to switch between Q&A and Summarization.
2. Visual Loading Status via a dedicated status box and progress bars.

# **1. Setup & Installation**

In [1]:
# Install both UI and Document Intelligence libraries
!pip install -q pymupdf gradio llama-index llama-index-readers-file google-generativeai jedi

# **2. Main Application (Frontend & Backend Logic)**

In [2]:
#Frontend & Backend Logic in single codeblock,
# so when code block run the entire app launches at once to prevent "Variable Not Defined" errors.

# PRE-REQUISITE: Run this in a cell first: !pip install pymupdf

import gradio as gr
import time
import fitz  # PyMuPDF
import re

# --- 1. LOGIC FUNCTIONS ---

# Global variable to store combined text from all uploaded PDFs
document_memory = ""

def process_pdf(files):
    """Extracts text from multiple PDFs and merges them into document_memory."""
    global document_memory
    if not files:
        return "⚠️ Error: No files detected. Please upload one or more PDFs."

    document_memory = "" # Reset memory for new batch
    total_pages = 0
    file_names = []

    try:
        for file in files:
            doc = fitz.open(file.name)
            total_pages += len(doc)
            file_names.append(file.name.split('/')[-1])
            for page in doc:
                document_memory += page.get_text()

        if not document_memory.strip():
            return "⚠️ Warning: The uploaded files appear to be empty or image-based scans."

        return (f"✅ SUCCESS: Indexed {len(files)} files ({total_pages} total pages).\n"
                f"Documents: {', '.join(file_names)}\n"
                f"Status: Ready for cross-document analysis.")
    except Exception as e:
        return f"❌ Error processing files: {str(e)}"

def handle_chat(message, history, mode):
    """Analyzes merged document_memory to answer questions."""
    global document_memory
    if not message:
        return "", history

    if not document_memory:
        bot_response = "I have no documents in my memory. Please upload PDFs and click 'Process'!"
    else:
        search_query = message.lower()

        if mode == "Summary":
            bot_response = f"Comprehensive Summary: Across the provided documents ({len(document_memory.split())} words), the text discusses: {document_memory[:300]}..."

        elif mode == "Key Takeaways":
            # Extract common themes (words longer than 6 chars)
            themes = list(set([w.strip(',.()').lower() for w in document_memory.split() if len(w) > 7]))
            bot_response = f"Primary themes identified across files: {', '.join(themes[:5])}."

        else:
            # Q&A Logic: Sentence retrieval
            sentences = re.split(r'(?<=[.!?]) +', document_memory)
            matching_sentences = [s.strip() for s in sentences if search_query in s.lower()]

            if matching_sentences:
                # Provide up to 3 contextually relevant sentences
                context = " ".join(matching_sentences[:3])
                bot_response = f"From the documents: \"{context}\""
            else:
                bot_response = f"I couldn't find a direct mention of '{message}' in the combined text. Try another keyword."

    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": bot_response})
    return "", history

# --- 2. THEME & STYLING ---

monochrome_theme = gr.themes.Soft(
    primary_hue="slate",
    secondary_hue="gray",
    neutral_hue="slate",
).set(
    button_primary_background_fill="*neutral_900",
    button_primary_text_color="white",
    block_title_text_weight="700",
)

custom_css = """
#status_box { background-color: #f8fafc; border: 1px solid #cbd5e1; }
#chat_win { border-radius: 10px; }
"""

# --- 3. UI LAYOUT ---

gr.close_all() # Reset any existing loops

with gr.Blocks(theme=monochrome_theme, css=custom_css, title="Multi-PDF Research Buddy") as demo:
    gr.Markdown("# 📑 Multi-PDF Research Buddy")
    gr.Markdown("### *Analyze multiple research papers simultaneously in a minimalist workspace.*")

    with gr.Row():
        # LEFT COLUMN
        with gr.Column(scale=2, min_width=350):
            gr.Markdown("#### ⚙️ Document Controls")
            # EXTRA FEATURE: Multiple file support
            pdf_input = gr.File(
                label="Upload PDFs",
                file_types=[".pdf"],
                file_count="multiple" # This enables multi-file upload
            )

            mode_select = gr.Dropdown(
                choices=["Standard Q&A", "Summary", "Key Takeaways"],
                value="Standard Q&A",
                label="Analysis Mode"
            )

            process_btn = gr.Button("🔄 Process All Documents", variant="primary")

            status_output = gr.Textbox(
                label="System Status",
                lines=6,
                elem_id="status_box",
                interactive=False
            )

            exit_btn = gr.Button("🚪 Shut Down Portal", variant="stop")

        # RIGHT COLUMN
        with gr.Column(scale=3):
            gr.Markdown("#### 💬 Chat Interface")
            chatbot = gr.Chatbot(
                label="Analysis History",
                height=500,
                type="messages",
                allow_tags=False,
                elem_id="chat_win"
            )

            user_input = gr.Textbox(placeholder="Query your documents...", label="Your Question")

            with gr.Row():
                send_btn = gr.Button("📤 Send", variant="primary")
                clear_btn = gr.Button("🗑️ Clear Chat History")

    # --- 4. EVENT LISTENERS ---

    process_btn.click(fn=process_pdf, inputs=pdf_input, outputs=status_output)

    send_btn.click(handle_chat, inputs=[user_input, chatbot, mode_select], outputs=[user_input, chatbot])
    user_input.submit(handle_chat, inputs=[user_input, chatbot, mode_select], outputs=[user_input, chatbot])

    clear_btn.click(lambda: [], None, chatbot, queue=False)

    exit_btn.click(lambda: gr.Info("Session Terminated."), None, None).then(fn=demo.close)

# --- 5. LAUNCH ---
if __name__ == "__main__":
    demo.launch(debug=True, share=True)

/tmp/ipython-input-1995458098.py:99: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=monochrome_theme, css=custom_css, title="Multi-PDF Research Buddy") as demo:
/tmp/ipython-input-1995458098.py:99: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=monochrome_theme, css=custom_css, title="Multi-PDF Research Buddy") as demo:
/tmp/ipython-input-1995458098.py:134: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5954586a0ae01895a8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5954586a0ae01895a8.gradio.live
